In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
NER = spacy.load("en_core_web_lg")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv('amazon_alexa.tsv', sep='\t')

df.head()


[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [WinError 10054] An existing connection was forcibly
[nltk_data]     closed by the remote host>
[nltk_data] Error loading tagsets: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading omw-1.4: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [3]:
df = df.drop(['rating', 'date','feedback' ,'variation'], axis=1)

In [4]:
def clean_text(review):
    stopword = set(stopwords.words('english'))
    stopword.add('')
    review = word_tokenize(review)
    review = [rev.lower() for rev in review]
    review = [re.sub("[^A-Za-z0-9]",' ',rev) for rev in review]
    review = [re.sub(r'[0-9]+',' ',rev) for rev in review]
    review = " ".join(WordNetLemmatizer().lemmatize(i) for i in review)
    
    return review

df['verified_reviews'] = df['verified_reviews'].apply(lambda rev: clean_text(rev))

df

,verified_reviews
0,love my echo
1,loved it
2,sometimes while playing a game you can answe...
3,i have had a lot of fun with this thing my ...
4,music
...,...
3145,perfect for kid adult and everyone in betwee...
3146,listening to music searching location chec...
3147,i do love these thing i have them running my...
3148,only complaint i have is that the sound qualit...


In [5]:
df['verified_reviews_POS'] = df['verified_reviews'].apply(lambda x: nltk.pos_tag(x.split()))
df


,verified_reviews,verified_reviews_POS
0,love my echo,"[(love, VB), (my, PRP$), (echo, NN)]"
1,loved it,"[(loved, VBD), (it, PRP)]"
2,sometimes while playing a game you can answe...,"[(sometimes, RB), (while, IN), (playing, VBG),..."
3,i have had a lot of fun with this thing my ...,"[(i, NN), (have, VBP), (had, VBN), (a, DT), (l..."
4,music,"[(music, NN)]"
...,...,...
3145,perfect for kid adult and everyone in betwee...,"[(perfect, NN), (for, IN), (kid, NN), (adult, ..."
3146,listening to music searching location chec...,"[(listening, VBG), (to, TO), (music, NN), (sea..."
3147,i do love these thing i have them running my...,"[(i, NN), (do, VBP), (love, VB), (these, DT), ..."
3148,only complaint i have is that the sound qualit...,"[(only, RB), (complaint, NN), (i, NNS), (have,..."


In [6]:
dictionary = {}

In [7]:
for rev in df['verified_reviews_POS']:
    for word in rev:
        if word[1] in dictionary:
            #if word[0] not in word[1]:
            dictionary[word[1]].add(word[0])
        else:
            dictionary[word[1]] = set(word[0])


In [8]:
for key in dictionary:
    dictionary[key] = list(dictionary[key])

In [9]:
# print(dictionary)

In [10]:
out = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dictionary.items() ]))
out

,VB,PRP$,NN,VBD,PRP,RB,IN,VBG,DT,MD,...,PDT,JJR,JJS,WDT,RBR,RBS,EX,WP$,UH,NNP
0,gb,your,half,owned,i,pleasantly,amazon,amazon,half,should,...,half,e,s,h,e,best,e,e,h,e
1,simplify,his,shop,kinda,s,specifically,find,linking,the,button,...,hole,controller,m,wa,more,t,h,h,o,x
2,control,weather,engine,posed,he,then,text,liking,another,would,...,all,lyric,est,c,better,s,t,w,NaN,b
3,connect,m,sounding,arrived,us,defently,since,getting,all,c,...,wa,longer,youngest,t,le,m,there,s,NaN,r
4,learn,her,ceiling,red,him,flawlessly,except,tipping,bluetooth,ca,...,hair,younger,le,wherever,wa,o,r,o,NaN,kitchen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,NaN,NaN,ghz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1793,NaN,NaN,note,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1794,NaN,NaN,receive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1795,NaN,NaN,dictionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
!pip install xlwt
out.to_excel('postagging.xlsx')

In [ ]:
df['verified_reviews_NER'] = df['verified_reviews'].apply(lambda x: NER(x))

In [ ]:
df['verified_reviews_NER']

In [ ]:
dict_ner = {}
for rev in df['verified_reviews_NER']:
    for word in rev.ents:
        if word.label_ in dict_ner:
            dict_ner[word.label_].add(word.text)
        else:
            dict_ner[word.label_] = set(word.text)
            
dict_ner

        

In [ ]:
for key in dict_ner:
    dict_ner[key] = list(dict_ner[key])

In [ ]:
out_ner = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_ner.items() ]))
out_ner

In [ ]:
out_ner.to_excel('ner.xlsx')

In [ ]:
dict_dep = { 'Token': [], 'Relation':[],'Head':[],'Children':[]}
            

In [ ]:
for rev in df['verified_reviews_NER']:
    for token in rev:
        dict_dep['Token'].append(str(token.text))
        dict_dep['Relation'].append(str(token.dep_))
        dict_dep['Head'].append(str(token.head.text))
        dict_dep['Children'].append(str([child for child in token.children]))

            
out_dep = pd.DataFrame(dict_dep)
out_dep.head(-10)

In [ ]:
out_dep.to_excel('dep.xlsx')